# Loading

In [158]:
import json.decoder

import numpy as np
import pandas as pd
import sklearn as sk
import json as json
# import seaborn as sns
import matplotlib
import pickle as pk
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('json version:', json.__version__)
print('matplotlib version:', matplotlib.__version__)

numpy version: 1.23.3
pandas version: 1.5.0
scikit-learn version: 1.1.2
json version: 2.0.9
matplotlib version: 3.6.0


In [159]:
#DataSets

# df = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv')
RAW_recipes = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\RAW_recipes.csv")
RAW_interactions = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\RAW_interactions.csv")
pkl_recipes = pd.read_pickle(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\ingr_map.pkl")

pp_recipes = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\PP_recipes.csv")
pp_users = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\PP_users.csv")

final_ingredients_Raw = pd.read_csv(r"C:\Users\evald\PycharmProjects\IndividualProject\set\results.csv")

In [160]:
final_ingredients = final_ingredients_Raw[:8000]

Setting data

In [161]:
#Store average rating of meals
AvgInteractions = RAW_interactions['rating'].mean()

#Group seperate meal ratings together and get average meal score equally prevent recipe_id from being an index
newRatingSet = RAW_interactions.groupby(['recipe_id'])['rating'].agg(['mean', 'count'])
print(newRatingSet.head(10))
newRatingSet.reset_index(inplace=True)

#Rename columns
newRatingSet.rename(columns={'mean': 'average_rating', 'count': 'votes'}, inplace=True)
newRatingSet.head(10)
#Merge ratings dataframe with recipe Dataframe
food_df = pd.merge(RAW_recipes, newRatingSet, left_on='id', right_on='recipe_id')

               mean  count
recipe_id                 
38         4.250000      4
39         3.000000      1
40         4.333333      9
41         4.500000      2
43         1.000000      1
45         2.666667      3
46         5.000000      2
48         1.000000      2
49         4.350000     20
50         4.500000      2


# Machine learning

## Load libraries

In [162]:
from sklearn import preprocessing,metrics,neighbors,model_selection
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
from scipy.sparse import csr_matrix

from sklearn.preprocessing import Normalizer

## Hot Encoding

In [163]:
IngredientEncoding = final_ingredients[['id','name','ingredients']].__deepcopy__()
IngredientEncoding

,id,name,ingredients
0,424415,aromatic basmati rice rice cooker,"basmati rice, water, salt, cinnamon stick, gre..."
1,146223,pumpkin pie a la easy,"flour, oats, brown sugar, pecans, butter, eggs..."
2,312329,cheesy tomato soup with potatoes,"chicken broth, water, salt, black pepper, onio..."
3,74301,mini tacos,"wonton wrappers, hamburger, taco seasoning, sa..."
4,76272,rosemary s hanky panky s,"ground beef, ground sausage, velveeta cheese, ..."
...,...,...,...
7995,210151,deviled sausages,"white wine, soy sauce, malt vinegar, worcester..."
7996,413579,butternut squash pesto pizza flatbread,"butternut squash, onion, brown sugar, sea salt..."
7997,93183,strawberry tossed salad,"vegetable oil, sugar, cider vinegar, garlic, s..."
7998,135158,byessar moroccan broad bean dip,"broad bean, lemon rind, garlic cloves, ras el ..."


In [164]:
IngredientEncoding_recipe =  final_ingredients[['id','name','description','ingredients']].__deepcopy__()
favorite_ingredients = ['dressing', 'sausage', 'chile', 'cake mix', 'cheddar', 'tortilla','pickle', 'lettuce', 'cream cheese', 'rum']

IngredientEncoding_recipe[['dressing', 'sausage', 'chile', 'cake mix', 'cheddar', 'tortilla','pickle', 'lettuce', 'cream cheese', 'rum']] = 0

## Tokenization

In [165]:
# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')
# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(IngredientEncoding['ingredients'])
# Print the shape of the tfidf_matrix
tfidf_matrix.shape

(8000, 1955)

## Cosine similarity

In [166]:
# Compute the cosine similarity between each movie description
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [174]:
indices = pd.Series(IngredientEncoding.index, index=IngredientEncoding['ingredients']).drop_duplicates()

In [179]:
IngredientEncoding

,id,name,ingredients
0,424415,aromatic basmati rice rice cooker,"basmati rice, water, salt, cinnamon stick, gre..."
1,146223,pumpkin pie a la easy,"flour, oats, brown sugar, pecans, butter, eggs..."
2,312329,cheesy tomato soup with potatoes,"chicken broth, water, salt, black pepper, onio..."
3,74301,mini tacos,"wonton wrappers, hamburger, taco seasoning, sa..."
4,76272,rosemary s hanky panky s,"ground beef, ground sausage, velveeta cheese, ..."
...,...,...,...
7995,210151,deviled sausages,"white wine, soy sauce, malt vinegar, worcester..."
7996,413579,butternut squash pesto pizza flatbread,"butternut squash, onion, brown sugar, sea salt..."
7997,93183,strawberry tossed salad,"vegetable oil, sugar, cider vinegar, garlic, s..."
7998,135158,byessar moroccan broad bean dip,"broad bean, lemon rind, garlic cloves, ras el ..."


In [175]:
def get_recommendations(title, cosine_sim=cosine_sim, num_recommend = 10):
    idx = indices[title]# Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))# Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)# Get the scores of the 10 most similar movies
    top_similar = sim_scores[1:num_recommend+1]# Get the movie indices
    movie_indices = [i[0] for i in top_similar]# Return the top 10 most similar movies
    return IngredientEncoding['ingredients'].iloc[movie_indices]

In [178]:
get_recommendations('Chicken', num_recommend = 20)

KeyError: 'Chicken'

# Crosstabulation

In [ ]:
# crossTab = pd.crosstab(RAW_recipes["replaced"],final_ingredients["ingredients"]).sample(1000)

## Numerical collumns only

In [ ]:
Recipe_prep = final_ingredients[["id","minutes","n_steps","n_ingredients","average_rating","votes","like"]]

In [ ]:
Recipe_prep

## Merging important feature tags

In [ ]:

def import_feature_tags():
    data = IngredientEncoding[:4000]
    data['ingredients']= data['ingredients'].str.replace(']', '',regex=False)
    data['ingredients']= data['ingredients'].str.replace('[', '',regex=False)
    for i in range(0,data.shape[0]):
        data["ift"] = data["name"]+' '+ data["ingredients"]
    return data

In [ ]:
data = import_feature_tags()

In [ ]:
data.head()

## Vectorization

In [ ]:
countVec = CountVectorizer

TfidVec

In [ ]:
vector = TfidfVectorizer()
vectors = vector.fit_transform(data["ift"].apply(lambda x: np.str_(x)))

In [ ]:
vectors.shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
similarity.shape
# Collums represent ids
similarity[8][8]

In [ ]:
data['ingredients'][2]

In [ ]:
scoresTemp = []
def recommend(name):
    recipie_id=data[data['ingredients'].str.contains(name)]["id"].values[0]
    scores = list(enumerate(similarity[recipie_id]))
    sorted_scores = sorted(scores,key=lambda x:x[1],reverse=True)
    sorted_scores=sorted_scores[1:]
    scoresTemp.append(sorted_scores)
    # print(sorted_scores +" " + " scores")
    # print(recipie_id)
    recipes = [data[recipes[0]==data['id']]['name'].values[0] for recipes in sorted_scores]
    # recipes = [data[recipes[0]==data['ids'] , " " , recipes[1]]['name'].values[0] for recipes in sorted_scores]
    # print(recipes)
    return recipes

In [ ]:
def recomend_ten(recipe_list):
    ten=[]
    count=0
    print("----" * 8)
    print("Top 10 recipes")
    print("----" * 8)
    for recipe in recipe_list:
        if count > 9:
            break
        count +=1
        print(recipe, ":" )
        ten.append(recipe)
    return ten

In [ ]:
r_list = recommend('flour')

In [ ]:
re = recomend_ten(r_list)

In [ ]:
scoresTemp

In [ ]:
re

# KNN algorithm


In [ ]:
popularity_threshold = 30
rating_popular_recipe=final_ingredients.where(final_ingredients.votes >= popularity_threshold).dropna()
rating_popular_recipe.head(10)

In [ ]:
rating_popular_recipe.shape

In [ ]:
recipePrepSet = Recipe_prep[:3765]
recipePrepSet

In [ ]:
recipe_like_features_df = rating_popular_recipe.pivot_table(index='name',columns='ingredient_ids',values='like').fillna(0)
recipe_like_features_df.head()

In [ ]:
recipe_features_df = rating_popular_recipe.pivot_table(index='name',columns='id',values='average_rating').fillna(0)
recipe_features_df.head()

In [ ]:
recipe_features_df_matrix = csr_matrix(recipe_features_df.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(recipe_features_df_matrix)

In [ ]:
recipe_features_df.shape

Selecting random index

In [ ]:
query_index = np.random.choice(recipe_features_df.shape[0])
# query_index = recipe_features_df.shape[4]
print(query_index)
distances, indices = model_knn.kneighbors(recipe_features_df.iloc[query_index,:].values.reshape(1,-1), n_neighbors= 5)

In [ ]:
recipe_features_df.head(10)

In [ ]:
indices.flatten()

In [ ]:
distances.flatten()

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(recipe_features_df.index[query_index]))
    else:
        print('{0}: {1}:, with distance of {1}:\n'.format(i, recipe_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

# Decission tree

Plot the knn

In [ ]:
from sklearn.model_selection import train_test_split
X = Recipe_prep[['average_rating']]
y = Recipe_prep[['like']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

In [ ]:
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.linear_model import LinearRegression

multiReg = LinearRegression()

In [ ]:
lin_score = cross_val_score(multiReg,X_train,y_train,cv=5,scoring='neg_mean_absolute_error')

In [ ]:
lin_score_n = -lin_score
lin_score_n

In [ ]:
np.mean(lin_score_n)

Decission tree Classifier

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree_clf = DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_split=2, random_state = 42)
tree_clf.fit(X_train,y_train)

In [ ]:
y_predict = tree_clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))

In [ ]:
tree.plot_tree(tree_clf)

### Metrics error score

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_predict))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_predict))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_predict)))

In [ ]:
# def knn_comparison(data, k):
#     x = data[[‘X’,’Y’]].values
#     y = data[‘class’].astype(int).values
#     clf = neighbors.KNeighborsClassifier(n_neighbors=k)
#     clf.fit(x, y)
#     # Plotting decision region
#     plot_decision_regions(x, y, clf=clf, legend=2)
#     # Adding axes annotations
#     plt.xlabel(‘X’)
#     plt.ylabel(‘Y’)
#     plt.title(‘Knn with K=’+ str(k))
#     plt.show()

In [ ]:
# data1 = final_ingredients
# for i in [1,5,20,30,40,80]:
#     knn_comparison(data1, i)

### Train knn

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn_classifier.fit(X_train,y_train)

In [ ]:
y_pred = knn_classifier.predict(X_test)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Plotting knn

In [ ]:
xChoice = Recipe_prep.__deepcopy__()
xChoice = xChoice.where(Recipe_prep['average_rating'] > 3)
# xChoice = recipesLikesDislikes[['average_rating']]
yChoice = Recipe_prep.__deepcopy__()
yChoice = Recipe_prep[(Recipe_prep['like'] == 1)]

xChoice.shape




In [ ]:
yChoice.shape

In [ ]:
xChoice.dropna()

In [ ]:
empty_count = xChoice.isnull().sum()
empty_count

In [ ]:
yChoice.dropna()

In [ ]:
tree_clf = DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_split=2, random_state = 42)
tree_clf.fit(xChoice,yChoice)

tree.plot_tree(tree_clf);